In [1]:
from selenium import webdriver
from time import sleep, strftime
import pause
import html_to_json
import pandas as pd
import numpy as np
from datetime import datetime as dt
from itertools import chain
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
import sqlite3
import os
import re
from IPython.display import display

In [2]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pretty_html_table import build_table

def send_mail(subject, body):
    
    message = MIMEMultipart()
    message['Subject'] = 'PM_' + subject
    message['From'] = 'russele7oge@gmail.com'
    message['To'] = 'russele7oge@gmail.com'

    body_content = build_table(body, 'blue_light')
    message.attach(MIMEText(body_content, "html"))
    msg_body = message.as_string()

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(message['From'], 'vmujqiiclzpfdlxu')
    server.sendmail(message['From'], message['To'], msg_body)
    server.quit()

In [3]:
import sys
# appending a path
sys.path.append(r'C:\Users\acer\Desktop\RUSLAN\PM')
from classes_opt import *

In [4]:
CHAMPS = list(data_list.keys())
display(CHAMPS)

['england',
 'germany',
 'spain',
 'italy',
 'france',
 'belgium',
 'netherlands',
 'portugal',
 'turkey',
 'czech',
 'poland',
 'greece',
 'scotland',
 'swiss']

In [13]:
# CHAMPS = ['england',
#  'germany',
#  'spain',
#  'italy',
#  'france',
#  'belgium',
#  'netherlands',
#  'portugal']

In [5]:
END_DAY_TIME = dt.now().replace(hour = 23, minute = 55, second = 0, microsecond = 0)
display(f'END_DAY_TIME = {END_DAY_TIME}')

'END_DAY_TIME = 2022-01-14 23:55:00'

# 1. ПАРСИНГ РАСПИСАНИЯ ИГР

In [6]:
data_champ_list = []
for elem in CHAMPS:
    print(elem)    
    try:
        page = sport_page(COUNTRY = elem, TOURNAMENT = data_list[elem][0])
        page.download_webpage(URL = data_list[elem][1])
        data = page.df_games
        data.reset_index(inplace = True)
        data.rename(columns = {'index':'GAME_ID'}, inplace = True)
        data_champ_list.append(data)
    except:
        aa = dt.now()
        print(f"ERROR with download <{elem}> / time = {aa}")
        with open('errors.txt', 'a+') as file:
            file.write(f"ERROR with download <{elem}> / time = {aa}")
            file.close()
        del aa

england
ERROR with download <england> / time = 2022-01-14 14:06:23.374923
germany
ERROR with download <germany> / time = 2022-01-14 14:06:26.780362
spain
ERROR with download <spain> / time = 2022-01-14 14:06:30.318481
italy
ERROR with download <italy> / time = 2022-01-14 14:06:33.910369
france
ERROR with download <france> / time = 2022-01-14 14:06:37.546037
belgium
ERROR with download <belgium> / time = 2022-01-14 14:06:41.084585
netherlands
ERROR with download <netherlands> / time = 2022-01-14 14:06:44.512851
portugal
ERROR with download <portugal> / time = 2022-01-14 14:06:47.996161
turkey
ERROR with download <turkey> / time = 2022-01-14 14:06:51.478240
czech
ERROR with download <czech> / time = 2022-01-14 14:06:55.069494
poland
ERROR with download <poland> / time = 2022-01-14 14:06:58.569425
greece
ERROR with download <greece> / time = 2022-01-14 14:07:02.054431
scotland
ERROR with download <scotland> / time = 2022-01-14 14:07:05.631457
swiss
ERROR with download <swiss> / time = 202

In [34]:
data = pd.concat(data_champ_list).copy()
print('parced data')
data.to_excel(f'data_parced.xlsx')
display(data)

parced data


,GAME_ID,GAME_DT,OBSDT,HT,GT,COUNTRY,TOURNAMENT,HW_COEF,DR_COEF,GW_COEF
0,0,2021-12-26 15:30:00,2021-12-22 23:08:18,Вулверхэмптон,Уотфорд,england,ПРЕМЬЕР-ЛИГА,1.82,3.55,4.50
1,1,2021-12-26 15:30:00,2021-12-22 23:08:18,Ливерпуль,Лидс,england,ПРЕМЬЕР-ЛИГА,1.14,8.80,15.50
2,2,2021-12-26 18:00:00,2021-12-22 23:08:18,Манчестер Сити,Лестер,england,ПРЕМЬЕР-ЛИГА,1.19,7.30,13.00
3,3,2021-12-26 18:00:00,2021-12-22 23:08:18,Норвич,Арсенал Лондон,england,ПРЕМЬЕР-ЛИГА,7.10,4.60,1.44
4,4,2021-12-26 18:00:00,2021-12-22 23:08:18,Тоттенхэм,Кристал Пэлас,england,ПРЕМЬЕР-ЛИГА,1.79,3.65,4.50
...,...,...,...,...,...,...,...,...,...,...
1,1,2021-12-26 18:00:00,2021-12-22 23:12:09,Данди Юнайтед,Хиберниан,scotland,ПРЕМЬЕРШИП,2.90,3.30,2.46
2,2,2021-12-26 18:00:00,2021-12-22 23:12:09,Абердин,Данди ФК,scotland,ПРЕМЬЕРШИП,1.64,3.95,5.30
3,3,2021-12-26 18:00:00,2021-12-22 23:12:09,Хартс,Росс Каунти,scotland,ПРЕМЬЕРШИП,1.55,4.20,5.90
4,4,2021-12-26 18:00:00,2021-12-22 23:12:09,Рейнджерс,Сент-Миррен,scotland,ПРЕМЬЕРШИП,1.17,7.50,16.50


In [35]:
data[~(data['HT'].apply(lambda x:  'Хозяева' in x )) & 
     ~(data['GT'].apply(lambda x:  'Хозяева' in x ))]

,GAME_ID,GAME_DT,OBSDT,HT,GT,COUNTRY,TOURNAMENT,HW_COEF,DR_COEF,GW_COEF
0,0,2021-12-26 15:30:00,2021-12-22 23:08:18,Вулверхэмптон,Уотфорд,england,ПРЕМЬЕР-ЛИГА,1.82,3.55,4.50
1,1,2021-12-26 15:30:00,2021-12-22 23:08:18,Ливерпуль,Лидс,england,ПРЕМЬЕР-ЛИГА,1.14,8.80,15.50
2,2,2021-12-26 18:00:00,2021-12-22 23:08:18,Манчестер Сити,Лестер,england,ПРЕМЬЕР-ЛИГА,1.19,7.30,13.00
3,3,2021-12-26 18:00:00,2021-12-22 23:08:18,Норвич,Арсенал Лондон,england,ПРЕМЬЕР-ЛИГА,7.10,4.60,1.44
4,4,2021-12-26 18:00:00,2021-12-22 23:08:18,Тоттенхэм,Кристал Пэлас,england,ПРЕМЬЕР-ЛИГА,1.79,3.65,4.50
...,...,...,...,...,...,...,...,...,...,...
1,1,2021-12-26 18:00:00,2021-12-22 23:12:09,Данди Юнайтед,Хиберниан,scotland,ПРЕМЬЕРШИП,2.90,3.30,2.46
2,2,2021-12-26 18:00:00,2021-12-22 23:12:09,Абердин,Данди ФК,scotland,ПРЕМЬЕРШИП,1.64,3.95,5.30
3,3,2021-12-26 18:00:00,2021-12-22 23:12:09,Хартс,Росс Каунти,scotland,ПРЕМЬЕРШИП,1.55,4.20,5.90
4,4,2021-12-26 18:00:00,2021-12-22 23:12:09,Рейнджерс,Сент-Миррен,scotland,ПРЕМЬЕРШИП,1.17,7.50,16.50


In [30]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# END_DAY_TIME = END_DAY_TIME + relativedelta(days = 1)
# END_DAY_TIME

datetime.datetime(2021, 12, 23, 23, 55)

In [36]:
data = data[
      (data['GAME_DT'].apply(lambda x: x.toordinal()) <= END_DAY_TIME.toordinal())
    & ~(data['HT'].apply(lambda x:  'Хозяева' in x))  
    & ~(data['GT'].apply(lambda x:  'Гости' in x ))
].copy()
data.sort_values('GAME_DT', ascending = True, inplace = True)
data.reset_index(drop = True,inplace = True)
data['GAME_ID'] = data.index

In [32]:
if len(data) == 0:
    print()
    with open(f'report.txt', 'w') as f:
        f.write(f'TODAY IS NO GAMES _ {str(dt.today())}')
        f.close()
    sys.exit()

In [33]:
print('data daily')
display(data[['GAME_DT', 'HT', 'GT', 'COUNTRY', 'TOURNAMENT', 'HW_COEF', 'DR_COEF', 'GW_COEF']])
data.to_excel(f'data_daily.xlsx')

data daily


,GAME_DT,HT,GT,COUNTRY,TOURNAMENT,HW_COEF,DR_COEF,GW_COEF
0,2021-12-22 23:30:00,Атлетик Бильбао,Реал Мадрид,spain,ЛА ЛИГА,3.30,3.45,2.26
1,2021-12-23 17:00:00,Йени Малатьяспор,Кайсериспор,turkey,СУПЕРЛИГА,2.40,3.25,3.05
2,2021-12-23 20:00:00,Бешикташ,Гёзтепе,turkey,СУПЕРЛИГА,1.55,4.50,5.30
3,2021-12-23 20:45:00,ПСВ,Гоу Эхед Иглз,netherlands,ERIDIVISIE,1.14,8.30,17.50
4,2021-12-23 23:00:00,Спарта Роттердам,Валвейк,netherlands,ERIDIVISIE,1.80,3.60,4.60


In [30]:
data_interesting = data[(data['DR_COEF'] >= 3.) & (abs(data['HW_COEF'] - data['GW_COEF']) < 0.5 )][
    ['GAME_DT', 'HT', 'GT', 'COUNTRY', 'TOURNAMENT', 'HW_COEF', 'DR_COEF', 'GW_COEF']].copy()

print('data interesting')
display(data_interesting)

data_interesting.to_excel(f'data_interesting.xlsx')

data interesting


,GAME_DT,HT,GT,COUNTRY,TOURNAMENT,HW_COEF,DR_COEF,GW_COEF
0,2021-12-21 20:00:00,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,2.6,3.35,2.75
7,2021-12-21 23:30:00,Севилья,Барселона,spain,ЛА ЛИГА,2.8,3.20,2.75


# 2. Пересоздаем  df_week и  помещаем туда распарсенные данные

In [31]:
df_week = data.drop(columns = ['HW_COEF', 'DR_COEF', 'GW_COEF'])
df_week
for elem in lag_list:
    df_week[f'HWC_{elem[0]}'] = np.nan
    df_week[f'DRC_{elem[0]}'] = np.nan
    df_week[f'GWC_{elem[0]}'] = np.nan

In [32]:
print('df_week')
df_week.to_excel(f'df_week.xlsx')
display(df_week)

df_week


,GAME_ID,GAME_DT,OBSDT,HT,GT,COUNTRY,TOURNAMENT,HWC_3M,DRC_3M,GWC_3M,HWC_6M,DRC_6M,GWC_6M,HWC_30M,DRC_30M,GWC_30M,HWC_1H,DRC_1H,GWC_1H
0,0,2021-12-21 20:00:00,2021-12-21 19:00:26,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2021-12-21 20:00:00,2021-12-21 19:00:26,Алтай Измир,Трабзонспор,turkey,СУПЕРЛИГА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2021-12-21 20:45:00,2021-12-21 18:59:39,Витесс,Зволле,netherlands,ERIDIVISIE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2021-12-21 21:00:00,2021-12-21 18:58:00,Вильярреал,Алавес,spain,ЛА ЛИГА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2021-12-21 22:45:00,2021-12-21 18:58:24,Дженоа,Аталанта,italy,СЕРИЯ А,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,2021-12-21 22:45:00,2021-12-21 18:58:24,Ювентус,Кальяри,italy,СЕРИЯ А,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,2021-12-21 23:00:00,2021-12-21 18:59:39,АЗ Алкмар,Гронинген,netherlands,ERIDIVISIE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,2021-12-21 23:30:00,2021-12-21 18:58:00,Севилья,Барселона,spain,ЛА ЛИГА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
CHAMPS = list(df_week['COUNTRY'].unique())
CHAMPS

['turkey', 'netherlands', 'spain', 'italy']

# 3. Берем dt игр => вычисляем dt лагов для каждой игры

In [34]:
df_lag = data[['HT', 'GT', 'COUNTRY' , 'TOURNAMENT', 'GAME_DT']].copy()
for elem in lag_list:
    df_lag[f'LAG_DT_{elem[0]}'] = df_lag['GAME_DT'].apply(lambda x: x - elem[1])
df_lag.drop(columns = ['GAME_DT'], inplace= True)

In [35]:
df_lag = df_lag.melt(
    id_vars=['HT', 'GT', 'COUNTRY', 'TOURNAMENT'],
    value_vars=None,
    var_name='LAG_TYPE',
    value_name='LAG_DT',
    col_level=None)

In [36]:
print('df_lag')
df_lag.to_excel('df_lag.xlsx')
display(df_lag)

df_lag


,HT,GT,COUNTRY,TOURNAMENT,LAG_TYPE,LAG_DT
0,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,LAG_DT_3M,2021-12-21 19:57:00
1,Алтай Измир,Трабзонспор,turkey,СУПЕРЛИГА,LAG_DT_3M,2021-12-21 19:57:00
2,Витесс,Зволле,netherlands,ERIDIVISIE,LAG_DT_3M,2021-12-21 20:42:00
3,Вильярреал,Алавес,spain,ЛА ЛИГА,LAG_DT_3M,2021-12-21 20:57:00
4,Дженоа,Аталанта,italy,СЕРИЯ А,LAG_DT_3M,2021-12-21 22:42:00
5,Ювентус,Кальяри,italy,СЕРИЯ А,LAG_DT_3M,2021-12-21 22:42:00
6,АЗ Алкмар,Гронинген,netherlands,ERIDIVISIE,LAG_DT_3M,2021-12-21 22:57:00
7,Севилья,Барселона,spain,ЛА ЛИГА,LAG_DT_3M,2021-12-21 23:27:00
8,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,LAG_DT_6M,2021-12-21 19:54:00
9,Алтай Измир,Трабзонспор,turkey,СУПЕРЛИГА,LAG_DT_6M,2021-12-21 19:54:00


## 4. БЕРЕМ УНИКАЛЬНЫЕ ЗНАЧЕНИЯ времен для парсинга

In [37]:
lag_parse_time = df_lag.pivot_table(index = 'LAG_DT', columns = 'COUNTRY', values = 'HT', aggfunc = 'count').reset_index()
lag_parse_time.rename_axis(None, axis=1, inplace = True)
lag_parse_time.rename(columns = {'LAG_DT' : 'PARSE_DT'}, inplace = True)
print('lag_parse_time')
lag_parse_time.to_excel('lag_parse_time.xlsx')
display(lag_parse_time)

lag_parse_time


,PARSE_DT,italy,netherlands,spain,turkey
0,2021-12-21 19:00:00,NaN,NaN,NaN,2.0
1,2021-12-21 19:30:00,NaN,NaN,NaN,2.0
2,2021-12-21 19:45:00,NaN,1.0,NaN,NaN
3,2021-12-21 19:54:00,NaN,NaN,NaN,2.0
4,2021-12-21 19:57:00,NaN,NaN,NaN,2.0
5,2021-12-21 20:00:00,NaN,NaN,1.0,NaN
6,2021-12-21 20:15:00,NaN,1.0,NaN,NaN
7,2021-12-21 20:30:00,NaN,NaN,1.0,NaN
8,2021-12-21 20:39:00,NaN,1.0,NaN,NaN
9,2021-12-21 20:42:00,NaN,1.0,NaN,NaN


## 5. В ТЕЧЕНИЕ НЕДЕЛИ
## В цикле в момент dt_lag делаем парсинг коэффициентов и помещаем их в df_week

In [38]:
def fill_the_week(dpage, dlag, df_week_in):    
    df_slice = dpage.merge(dlag, how = 'left', on = ['HT', 'GT', 'TOURNAMENT'])
    df_slice['DELTA_SEC'] = (df_slice['OBSDT'] - df_slice['LAG_DT']).apply(lambda x: x.total_seconds())
    print('df_slice BEFORE CUT [DELTA_SEC] < 2 minutes')
    display(df_slice)    
    df_slice = df_slice[
        (df_slice['DELTA_SEC'] < 60*2.5)      # 60 * 2.5
#         & (df_slice['DELTA_SEC'] >= 0)   # 0
    ].reset_index(drop = True)
    print('df_slice AFTER CUT [DELTA_SEC] < 2 minutes')
    display(df_slice)
    # Заполняем weekly значениями из slice
    df_week_out = df_week_in.copy()
    for j in range(len(df_slice)):
        row = df_slice.loc[[j]]
        pref = row['LAG_TYPE'].values[0][7:]
        df_week_out.loc[
            ( 
                (df_week_out['HT']  == row['HT'].values[0])   
              & (df_week_out['GT']  == row['GT'].values[0])
              & (df_week_out['TOURNAMENT']  == row['TOURNAMENT'].values[0])
            ),[
                f'HWC_{pref}', f'DRC_{pref}', f'GWC_{pref}']] = row[['HW_COEF', 'DR_COEF', 'GW_COEF']].values 
#     display(df_week_out)
    return df_week_out

In [39]:
data_interesting = data[(data['DR_COEF'] >= 3.) & (abs(data['HW_COEF'] - data['GW_COEF']) < 0.5 )][
    ['GAME_DT', 'HT', 'GT', 'COUNTRY', 'TOURNAMENT', 'HW_COEF', 'DR_COEF', 'GW_COEF']].copy()

In [40]:
i = 0

In [41]:
df_int = df_week[
    (df_week['DRC_3M'] >= 3.) 
    & (abs(df_week['HWC_3M'] - df_week['GWC_3M']) <= 0.15)
    & ((df_week['GAME_DT'] - lag_parse_time.loc[i]['PARSE_DT']).apply(lambda x: x.total_seconds()) < 60*2.5)
    & ((df_week['GAME_DT'] - lag_parse_time.loc[i]['PARSE_DT']).apply(lambda x: x.total_seconds()) > 0)
][['GAME_DT', 'HT', 'GT', 'COUNTRY', 'HWC_3M', 'DRC_3M', 'GWC_3M']].copy()
df_int

,GAME_DT,HT,GT,COUNTRY,HWC_3M,DRC_3M,GWC_3M


In [42]:
df_week

,GAME_ID,GAME_DT,OBSDT,HT,GT,COUNTRY,TOURNAMENT,HWC_3M,DRC_3M,GWC_3M,HWC_6M,DRC_6M,GWC_6M,HWC_30M,DRC_30M,GWC_30M,HWC_1H,DRC_1H,GWC_1H
0,0,2021-12-21 20:00:00,2021-12-21 19:00:26,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2021-12-21 20:00:00,2021-12-21 19:00:26,Алтай Измир,Трабзонспор,turkey,СУПЕРЛИГА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2021-12-21 20:45:00,2021-12-21 18:59:39,Витесс,Зволле,netherlands,ERIDIVISIE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2021-12-21 21:00:00,2021-12-21 18:58:00,Вильярреал,Алавес,spain,ЛА ЛИГА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2021-12-21 22:45:00,2021-12-21 18:58:24,Дженоа,Аталанта,italy,СЕРИЯ А,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,2021-12-21 22:45:00,2021-12-21 18:58:24,Ювентус,Кальяри,italy,СЕРИЯ А,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,2021-12-21 23:00:00,2021-12-21 18:59:39,АЗ Алкмар,Гронинген,netherlands,ERIDIVISIE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,2021-12-21 23:30:00,2021-12-21 18:58:00,Севилья,Барселона,spain,ЛА ЛИГА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
((df_week['GAME_DT'] - lag_parse_time.loc[i]['PARSE_DT']).apply(lambda x: x.total_seconds()) < 60*2.5)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
Name: GAME_DT, dtype: bool

In [43]:
# В цикле перебираем упорядоченные по времени начала моменты времени для парсинга 
for i in range(len(lag_parse_time)):    
    print(f"i = {i} / lag_parse_time = {lag_parse_time.loc[i]['PARSE_DT']}" )
    if dt.now() < lag_parse_time.loc[i]['PARSE_DT']:
#     if True:
        print(f'DO IF / current_time = {dt.now().replace(microsecond = 0)}')
        # Выжидаем до момента парсинга
        pause.until(lag_parse_time.loc[i]['PARSE_DT'])
        print(f'time after waiting = {dt.now().replace(microsecond = 0)}')

        # Выделяем чемпионаты в которых есть нужные игры по времени  PARSE_DT 
        bb = lag_parse_time.loc[i][CHAMPS].to_frame().reset_index()
        parse_list = list(bb[~bb[i].isna()]['index'])
        print(f'parse_list = {parse_list}')
        
         # Делаем парсинг
        data_champ_list2 = []
        for elem in parse_list:
            try:
                page2 = sport_page(COUNTRY = elem, TOURNAMENT = data_list[elem][0])
                page2.download_webpage(URL = data_list[elem][1])
                data2 = page2.df_games    
                data2.reset_index(inplace = True)
                data2.rename(columns = {'index':'GAME_ID'}, inplace = True)
                data_champ_list2.append(data2)
            except:
                aa = lag_parse_time.loc[i]['PARSE_DT']
                print(f"ERROR with download <{elem}> / time = {aa}")
                with open('errors.txt', 'a+') as file:
                    file.write(f"ERROR with download <{elem}> / time = {aa}")
                    file.close()
                del aa

        data2 = pd.concat(data_champ_list2).copy()
        data2 = data2.loc[data2['GAME_DT'] < END_DAY_TIME].copy()
        data2.reset_index(drop = True,inplace = True)
        data2.reset_index(inplace = True)
        data2.drop(columns = ['GAME_ID'], inplace = True)
        data2.rename(columns = {'index':'GAME_ID'}, inplace = True)    
        
        # Дополняем weekly спарсенными значениями
        df_week = fill_the_week(dpage = data2, dlag = df_lag, df_week_in = df_week)
        
        print('df_week')
        display(df_week)
        df_week.to_excel(f'df_week.xlsx')
        
          # Рассылка email интересных матчей        
        df_int = df_week[
                    (df_week['DRC_3M'] >= 3.) 
                    & (abs(df_week['HWC_3M'] - df_week['GWC_3M']) <= 0.15)
                    & ((df_week['GAME_DT'] - lag_parse_time.loc[i]['PARSE_DT']).apply(lambda x: x.total_seconds()) < 60*2.5)
                    & ((df_week['GAME_DT'] - lag_parse_time.loc[i]['PARSE_DT']).apply(lambda x: x.total_seconds()) > 0)
                        ][['GAME_DT', 'HT', 'GT', 'COUNTRY', 'HWC_3M', 'DRC_3M', 'GWC_3M']].copy()
        df_int.to_excel(f'df_int.xlsx')
        print('df_int')
        display(df_int)
        if (len(df_int) > 0) | True:
            send_mail(subject = '_'.join(df_int['COUNTRY'].to_list()), body = df_int)

i = 0 / lag_parse_time = 2021-12-21 19:00:00
i = 1 / lag_parse_time = 2021-12-21 19:30:00
DO IF / current_time = 2021-12-21 19:05:44
time after waiting = 2021-12-21 19:30:00
parse_list = ['turkey']
df_slice BEFORE CUT [DELTA_SEC] < 2 minutes


,GAME_ID,GAME_DT,OBSDT,HT,GT,COUNTRY_x,TOURNAMENT,HW_COEF,DR_COEF,GW_COEF,COUNTRY_y,LAG_TYPE,LAG_DT,DELTA_SEC
0,0,2021-12-21 20:00:00,2021-12-21 19:30:00,Алтай Измир,Трабзонспор,turkey,СУПЕРЛИГА,3.10,3.30,2.33,turkey,LAG_DT_3M,2021-12-21 19:57:00,-1620.0
1,0,2021-12-21 20:00:00,2021-12-21 19:30:00,Алтай Измир,Трабзонспор,turkey,СУПЕРЛИГА,3.10,3.30,2.33,turkey,LAG_DT_6M,2021-12-21 19:54:00,-1440.0
2,0,2021-12-21 20:00:00,2021-12-21 19:30:00,Алтай Измир,Трабзонспор,turkey,СУПЕРЛИГА,3.10,3.30,2.33,turkey,LAG_DT_30M,2021-12-21 19:30:00,0.0
3,0,2021-12-21 20:00:00,2021-12-21 19:30:00,Алтай Измир,Трабзонспор,turkey,СУПЕРЛИГА,3.10,3.30,2.33,turkey,LAG_DT_1H,2021-12-21 19:00:00,1800.0
4,1,2021-12-21 20:00:00,2021-12-21 19:30:00,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,2.44,3.35,2.90,turkey,LAG_DT_3M,2021-12-21 19:57:00,-1620.0
5,1,2021-12-21 20:00:00,2021-12-21 19:30:00,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,2.44,3.35,2.90,turkey,LAG_DT_6M,2021-12-21 19:54:00,-1440.0
6,1,2021-12-21 20:00:00,2021-12-21 19:30:00,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,2.44,3.35,2.90,turkey,LAG_DT_30M,2021-12-21 19:30:00,0.0
7,1,2021-12-21 20:00:00,2021-12-21 19:30:00,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,2.44,3.35,2.90,turkey,LAG_DT_1H,2021-12-21 19:00:00,1800.0


df_slice AFTER CUT [DELTA_SEC] < 2 minutes


,GAME_ID,GAME_DT,OBSDT,HT,GT,COUNTRY_x,TOURNAMENT,HW_COEF,DR_COEF,GW_COEF,COUNTRY_y,LAG_TYPE,LAG_DT,DELTA_SEC
0,0,2021-12-21 20:00:00,2021-12-21 19:30:00,Алтай Измир,Трабзонспор,turkey,СУПЕРЛИГА,3.10,3.30,2.33,turkey,LAG_DT_3M,2021-12-21 19:57:00,-1620.0
1,0,2021-12-21 20:00:00,2021-12-21 19:30:00,Алтай Измир,Трабзонспор,turkey,СУПЕРЛИГА,3.10,3.30,2.33,turkey,LAG_DT_6M,2021-12-21 19:54:00,-1440.0
2,0,2021-12-21 20:00:00,2021-12-21 19:30:00,Алтай Измир,Трабзонспор,turkey,СУПЕРЛИГА,3.10,3.30,2.33,turkey,LAG_DT_30M,2021-12-21 19:30:00,0.0
3,1,2021-12-21 20:00:00,2021-12-21 19:30:00,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,2.44,3.35,2.90,turkey,LAG_DT_3M,2021-12-21 19:57:00,-1620.0
4,1,2021-12-21 20:00:00,2021-12-21 19:30:00,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,2.44,3.35,2.90,turkey,LAG_DT_6M,2021-12-21 19:54:00,-1440.0
5,1,2021-12-21 20:00:00,2021-12-21 19:30:00,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,2.44,3.35,2.90,turkey,LAG_DT_30M,2021-12-21 19:30:00,0.0


df_week


...

,GAME_ID,GAME_DT,OBSDT,HT,GT,COUNTRY_x,TOURNAMENT,HW_COEF,DR_COEF,GW_COEF,COUNTRY_y,LAG_TYPE,LAG_DT,DELTA_SEC
0,0,2021-12-21 23:30:00,2021-12-21 23:27:00,Севилья,Барселона,spain,ЛА ЛИГА,2.5,3.15,3.15,spain,LAG_DT_3M,2021-12-21 23:27:00,0.0


df_week


,GAME_ID,GAME_DT,OBSDT,HT,GT,COUNTRY,TOURNAMENT,HWC_3M,DRC_3M,GWC_3M,HWC_6M,DRC_6M,GWC_6M,HWC_30M,DRC_30M,GWC_30M,HWC_1H,DRC_1H,GWC_1H
0,0,2021-12-21 20:00:00,2021-12-21 19:00:26,Адана Демирспор,Галатасарай,turkey,СУПЕРЛИГА,2.55,3.55,2.65,2.60,3.40,2.7,2.44,3.35,2.90,NaN,NaN,NaN
1,1,2021-12-21 20:00:00,2021-12-21 19:00:26,Алтай Измир,Трабзонспор,turkey,СУПЕРЛИГА,3.00,3.40,2.35,3.10,3.40,2.3,3.10,3.30,2.33,NaN,NaN,NaN
2,2,2021-12-21 20:45:00,2021-12-21 18:59:39,Витесс,Зволле,netherlands,ERIDIVISIE,1.57,4.10,5.90,1.57,4.10,5.9,1.55,4.10,6.00,1.59,3.95,5.80
3,3,2021-12-21 21:00:00,2021-12-21 18:58:00,Вильярреал,Алавес,spain,ЛА ЛИГА,1.41,4.80,8.40,1.41,4.80,8.4,1.39,5.00,8.60,1.41,4.80,8.40
4,4,2021-12-21 22:45:00,2021-12-21 18:58:24,Дженоа,Аталанта,italy,СЕРИЯ А,6.60,4.60,1.50,6.60,4.60,1.5,6.40,4.50,1.51,6.70,4.60,1.49
5,5,2021-12-21 22:45:00,2021-12-21 18:58:24,Ювентус,Кальяри,italy,СЕРИЯ А,1.22,6.50,17.00,1.22,6.40,17.0,1.23,6.00,17.00,1.21,6.40,19.00
6,6,2021-12-21 23:00:00,2021-12-21 18:59:39,АЗ Алкмар,Гронинген,netherlands,ERIDIVISIE,1.40,4.90,7.40,1.40,4.90,7.4,1.42,4.80,7.20,1.39,4.90,7.90
7,7,2021-12-21 23:30:00,2021-12-21 18:58:00,Севилья,Барселона,spain,ЛА ЛИГА,2.50,3.15,3.15,2.46,3.15,3.2,2.50,3.20,3.10,2.55,3.25,2.95


df_int


,GAME_DT,HT,GT,COUNTRY,HWC_3M,DRC_3M,GWC_3M


# КОНЕЦ НЕДЕЛИ
# 5. Парсим результаты игр


In [ ]:
print(f"CURRENT TIME IS {dt.now()}, SO LET'S WAIT FOR FINAL GAME ENDED")

In [ ]:
# Выжидаем окончание последней игры
if len(lag_parse_time) > 0:
    pause.until(lag_parse_time.loc[len(lag_parse_time)-1]['PARSE_DT'] + relativedelta(hours = 2.5))

In [ ]:
print(f"CURRENT TIME IS {dt.now()}, FINAL GAME IS ENDED. LET'S GET RESULTS")

In [ ]:
res_list = []
for elem in CHAMPS:
    print(elem)
    res_page = result_page(COUNTRY = elem, TOURNAMENT = data_list[elem][0])
    res_page.download_webpage(URL = data_list[elem][2])
    df_res = res_page.df_games
    df_res['HT'] = df_res['HT'].apply(lambda x: res_names_dict[elem][x] if x in res_names_dict[elem].keys() else x)
    df_res['GT'] = df_res['GT'].apply(lambda x: res_names_dict[elem][x] if x in res_names_dict[elem].keys() else x)
    res_list.append(df_res)

In [ ]:
df_res = pd.concat(res_list).copy()
print('df_res')
display(df_res)

#  6. Загружаем результаты в df_week

In [ ]:
df_week['DATE'] = df_week['GAME_DT'].dt.date

In [ ]:
df_week = df_week.merge(df_res[['HT', 'GT', 'DATE', 'RESULT']], how = 'left', on = ['HT', 'GT', 'DATE'])

In [ ]:
df_week['DELTA_GOALS'] = df_week.apply(
    lambda row: int(re.findall(r'\w*:', row['RESULT'])[0][:-1]) - int(re.findall(r':\w*', row['RESULT'])[0][1:])
                    if not pd.isnull(row['RESULT']) else np.nan, axis = 1) 

In [ ]:
df_week['RESULT_CODE'] = df_week.apply(lambda row:
                  'DR' if row['DELTA_GOALS'] == 0 else 
                      ('HW' if row['DELTA_GOALS'] > 0 else ('GW' if row['DELTA_GOALS'] < 0 else np.nan)), axis = 1)

In [ ]:
df_week = df_week[['GAME_ID',  'COUNTRY', 'TOURNAMENT', 'GAME_DT', 'HT', 'GT', 'RESULT', 'RESULT_CODE',
            'HWC_3M', 'DRC_3M', 'GWC_3M', 'HWC_6M', 'DRC_6M', 'GWC_6M',
           'HWC_30M', 'DRC_30M', 'GWC_30M'  ,'HWC_1H', 'DRC_1H', 'GWC_1H'
                  ]].copy()

In [ ]:
print('df_week with results')
display(df_week)
df_week.to_excel(f'df_week.xlsx')

# 7. Загружаем содержимое weekly_dataset в general_dataset

In [ ]:
# Сбросить df_general :
# bb = pd.DataFrame(columns = df_week.columns)
# bb.to_csv(f'df_general.csv', encoding = 'utf-8-sig')

In [ ]:
df_general = pd.read_excel(f'df_general.xlsx', index_col=0, encoding = 'utf-8-sig')
df_general

In [ ]:
ID_MAX = len(df_general)
display(ID_MAX)

In [ ]:
df_week['GAME_ID'] = df_week['GAME_ID'] + ID_MAX

In [ ]:
df_general = pd.concat([df_general, df_week]).reset_index(drop = True)
df_general.to_excel(f'df_general.xlsx', encoding = 'utf-8-sig')

In [ ]:
aa = pd.read_excel(f'df_general.excel', index_col=0, encoding = 'utf-8-sig')
print('general dataset after filling')
display(aa)
del aa